# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [16]:
# Install required packages
!pip install requests

In [17]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [18]:
import os
# This will delete the old database file if it exists
if os.path.exists("bitcoin_prices.db"):
    os.remove("bitcoin_prices.db")
    print("Old database deleted. A new one will be created.")
else:
    print("No old database found. A new one will be created.")

Old database deleted. A new one will be created.


In [19]:
# Read data from CoinGecko API for multiple currencies
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd,eur,gbp,jpy,cad,aud,chf,cnh,hkd,sgd,sek,nzd,krw,nok,inr,brl,rub,zar,mxn,myr,idr,try,sar,aed,thb"
response = requests.get(url)
data = response.json()

data

{'bitcoin': {'usd': 113357,
  'eur': 97347,
  'gbp': 84061,
  'jpy': 16718104,
  'cad': 157162,
  'aud': 175706,
  'chf': 91548,
  'hkd': 884164,
  'sgd': 145676,
  'sek': 1087842,
  'nzd': 192302,
  'krw': 157849392,
  'nok': 1166213,
  'inr': 9868565,
  'brl': 623488,
  'rub': 9156330,
  'zar': 2005898,
  'mxn': 2134699,
  'myr': 478708,
  'idr': 1846489703,
  'try': 4635081,
  'sar': 425381,
  'aed': 416362,
  'thb': 3696584}}

We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [20]:
# Function to convert USD to any currency using real exchange rates
def convert_usd_to_currency(usd_amount, target_currency, exchange_rates):
    if target_currency.upper() == 'USD':
        return usd_amount
    elif target_currency.upper() in exchange_rates:
        return round(usd_amount * exchange_rates[target_currency.upper()], 2)
    else:
        print(f"Currency {target_currency} not supported. Using USD.")
        return usd_amount

In [21]:
# Function to get real USD exchange rates
def get_exchange_rates():
    # Using a free API for exchange rates (alternative to CoinGecko's limited endpoint)
    exchange_url = "https://api.exchangerate-api.com/v4/latest/USD"
    try:
        response = requests.get(exchange_url)
        rates_data = response.json()
        return rates_data['rates']
    except:
        # Fallback rates if API fails
        print("Using fallback exchange rates")
        return {
            'EUR': 0.93, 'GBP': 0.80, 'JPY': 157.50, 'CAD': 1.36,
            'AUD': 1.50, 'CHF': 0.90, 'CNH': 7.25, 'HKD': 7.82,
            'SGD': 1.35, 'SEK': 10.50, 'NZD': 1.62, 'KRW': 1350.00,
            'NOK': 10.60, 'INR': 83.00, 'BRL': 5.40, 'RUB': 90.00,
            'ZAR': 18.50, 'MXN': 17.20, 'MYR': 4.70, 'IDR': 15500.00,
            'TRY': 32.00, 'SAR': 3.75, 'AED': 3.67, 'THB': 36.00
        }

In [22]:
# Get exchange rates
exchange_rates = get_exchange_rates()

# Extract USD price
usd_price = data["bitcoin"]["usd"]
timestamp = datetime.now().isoformat()

# Convert to multiple currencies
target_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CAD', 'AUD', 'ZAR']
converted_prices = {}

for currency in target_currencies:
    converted_prices[currency] = convert_usd_to_currency(usd_price, currency, exchange_rates)

print(f"Bitcoin price in USD: {usd_price}")
for currency, price in converted_prices.items():
    print(f"Price in {currency}: {price}")


Bitcoin price in USD: 113357
Price in USD: 113357
Price in EUR: 97146.95
Price in GBP: 83884.18
Price in JPY: 16748496.75
Price in CAD: 156432.66
Price in AUD: 174569.78
Price in ZAR: 1997350.34


In [23]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table with multiple currency columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT,
    usd REAL,
    eur REAL,
    gbp REAL,
    jpy REAL,
    cad REAL,
    aud REAL,
    zar REAL
)
''')

# Insert the new row with multiple currencies
cursor.execute('''
INSERT INTO prices (timestamp, usd, eur, gbp, jpy, cad, aud, zar)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', (
    timestamp,
    converted_prices['USD'],
    converted_prices['EUR'],
    converted_prices['GBP'],
    converted_prices['JPY'],
    converted_prices['CAD'],
    converted_prices['AUD'],
    converted_prices['ZAR']
))

conn.commit()
conn.close()


In [24]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()

('2025-08-19T19:29:44.669212', 113357.0, 97146.95, 83884.18, 16748496.75, 156432.66, 174569.78, 1997350.34)
